### SYK Hamiltonian

Sachdev–Ye–Kitaev (SYK) model, or SYK Hamiltonian is a system that is holographically dual to gravity. Successful teleportation is described from the dual gravity perspective as a particle traveling through a traversable wormhole. It is a sum of $q$ way interactions between Majorana fermions $\psi^j$, and the coefficients $J_j$ follows Gaussian distribution. 

![syk](images/fig_syk_hamiltonian.png)

However, experimentally implementing a full SYK model is extremely challenging. Therefore, the authors applied the machine learning approach to reduce the size of SYK Hamiltonian while preserving the gravitational physics. As a result, authors obtained a sparsified SYK Hamiltonian which has only five terms. The authors claim that the learned Hamiltonian reproduces gravitational dynamics of the SYK model.

### Majorana Fermions

In the previous section, authors reduced the $N=10$ dense SYK model to $N=7$ sparsified SYK model. To encode 7 Majorana fermions for the left black hole and 7 Majorana fermions for the right black hole, we require 7 qubits (two fermions per qubit). The authors decided to use standard Jordan-Wigner transformation to encode the Majorana fermions in the Hamiltonian. To make the decomposition of Majorana swap gates efficient, $\psi_L^1=ZX$, $\psi_L^2=ZY$, $\psi_R^1=X$, $\psi_R^2=Y$ are particularly chosen. The remaining transformations are chosen to satisfy $\psi_R^i\psi_L^j=\frac{i}{2}Z$ for shallow Trotterization of the coupling term $e^{i\mu V}$.

![majorana](images/fig_majorana.png)


### Dirac Fermions & SWAP gate

To swap a qubit into or out of the wormhole, authors paired up Majorana fermions on a single qubit into a Dirac fermion 
$$\chi_{L, R}=\frac{1}{\sqrt{2}}\left( \psi_{L, R}^1 + i \psi_{L, R}^2 \right) \\ \Rightarrow \chi_L=\frac{1}{2}(ZX+iZY), \ \chi_R=\frac{1}{2}(X+iY).$$
The swap gate is then given by
$$ \mathrm{SWAP} = \begin{bmatrix} 
\chi\chi^\dagger & \chi  \\ 
\chi & \chi^\dagger\chi
\end{bmatrix} $$
thus,
$$ \mathrm{SWAP}_{X+iY} = \begin{bmatrix} 
1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 \\ 
0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 
\end{bmatrix}, \quad \mathrm{SWAP}_{ZX+iZY} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & -1 & 0 \\
0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
\end{bmatrix}. $$ 
Since the $\mathrm{SWAP}_{X+iY}$ coincides with the standard swap gate, the final swap in the protocol from R to T (Fig. 1c of the paper) is replaced by direct measurement of the rightmost qubit of R to reduce gate count.

### Actual Codes

In [9]:
import pennylane as qml

worm_wires = list(range(7))
msg_wires = [7, 8]

wires = msg_wires + worm_wires
dev = qml.device('default.qubit', wires=len(wires), shots=None)

In [10]:
# Jordan-Wigner Transformation for Majorana Fermions
psi_map = {
    'L1': 'ZXIIIII',
    'L2': 'ZYIIIII',
    'L3': 'ZZZZZXI',
    'L4': 'ZZXIIII',
    'L5': 'ZZZZXII',
    'L6': 'ZZZXIII',
    'L7': 'ZZZZZZX',
    'R1': 'XIIIIII',
    'R2': 'YIIIIII',
    'R3': 'ZZZZZYI',
    'R4': 'ZZYIIII',
    'R5': 'ZZZZYII',
    'R6': 'ZZZYIII',
    'R7': 'ZZZZZZY'    
}

# q=4 Majorana basis for left system
basis_L = [
    ('L1', 'L2', 'L4', 'L5'), 
    ('L1', 'L3', 'L4', 'L7'), 
    ('L1', 'L3', 'L5', 'L6'), 
    ('L2', 'L3', 'L4', 'L6'), 
    ('L2', 'L3', 'L5', 'L7')
]

# q=4 Majorana basis for right system
basis_R = [
    ('R1', 'R2', 'R4', 'R5'), 
    ('R1', 'R3', 'R4', 'R7'), 
    ('R1', 'R3', 'R5', 'R6'), 
    ('R2', 'R3', 'R4', 'R6'), 
    ('R2', 'R3', 'R5', 'R7'),
]

# Majorana basis for coupling term
basis_V = [
    ('L1', 'R1'),
    ('L2', 'R2'),
    ('L3', 'R3'),
    ('L4', 'R4'),
    ('L5', 'R5'),
    ('L6', 'R6'),
    ('L7', 'R7'),
]

# Coefficient of sparsified SYK Hamiltonian
coeff = [-0.36, 0.19, -0.71, 0.22, 0.49]

In [11]:
def pauli_word(word):
    pw = qml.pauli.PauliWord({i: p for i, p in zip(worm_wires, word)})
    return pw

def mult_pauli_words(word_list, ignore_phase=False):
    assert len(word_list) > 1
    word, phase = pauli_word(word_list[0]), 1
    for w in word_list[1:]:
        word, new_phase = word * pauli_word(w)
        if not ignore_phase:
            phase = phase * new_phase
    return word, phase

def majorana_basis(psi, terms):
    word_list_list = [[psi[key] for key in term] for term in terms]
    basis_list = [mult_pauli_words(word_list) for word_list in word_list_list]
    return basis_list

def pauli_sentence(words, coeff, premult_i=False, mult=1.):
    ps = qml.pauli.PauliSentence({
        word: (1j*p if premult_i else p).real * c * mult
        for (word, p), c in zip(words, coeff)
    })
    return ps

In [12]:
m_L = majorana_basis(psi_map, basis_L)
ps_L = pauli_sentence(m_L, coeff, mult=0.25)

m_R = majorana_basis(psi_map, basis_R)
ps_R = pauli_sentence(m_R, coeff, mult=0.25)

m_V = majorana_basis(psi_map, basis_V)
ps_iV = pauli_sentence(m_V, [1/28]*7, premult_i=True, mult=0.5)

H_L = ps_L.hamiltonian(wire_order=worm_wires)
H_R = ps_R.hamiltonian(wire_order=worm_wires)
i_H_V = ps_iV.hamiltonian(wire_order=worm_wires)